In [ ]:
# Imports
from ast import literal_eval
from os import chdir, listdir, walk
import numpy as np
from IPython.display import clear_output

import dask.dataframe as dd
import pandas as pd
from dask.distributed import LocalCluster

In [2]:
# Instantiates dask cluster
cluster = LocalCluster()
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 63.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58009,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 63.63 GiB
Comm: tcp://127.0.0.1:58037,Total threads: 4
Dashboard: http://127.0.0.1:58040/status,Memory: 7.95 GiB
Nanny: tcp://127.0.0.1:58012,


In [3]:
# Move up two directories, to project base directory
chdir("..\\..")

# Gets all files in store_info subfolder
folders = listdir('data\\reviews\\')

In [4]:
# Instantiates empty list to hold file paths
paths = []

# Gets paths to review jsons by appending file name to name of containing folder
for dir, subfolders, items in walk('data\\reviews\\'):
    for item in items:
        paths.append(dir+'\\'+item)

In [5]:
def extract_reviews(row: pd.Series) -> pd.DataFrame:
    """Extracts and formats Pandas DataFrame of reviews from within json super-DataFrame.

    Args:
        row (pd.Series): Pandas Series corresponding to one json file.

    Returns:
        pd.DataFrame: Pandas DataFrame containing relevant contents of reviews json (i.e, excluding metadata).
    """

    # Parses review information as a dictionary (from string)
    dict_reviews = literal_eval(row["reviews"])

    # Parses review information as a pandas DataFrame
    df_reviews = pd.DataFrame(dict_reviews)
    
    # Adds appid to review DataFrame
    df_reviews["steam_appid"] = row["path"]

    # Extracts author information as separate DataFrame
    df_author_info = df_reviews["author"].apply(pd.Series)

    # Joins author information onto reviews DataFrame
    df_reviews = df_reviews.join(df_author_info.add_prefix("author_"))

    # Drops (now redundant) author information field
    df_reviews = df_reviews.drop("author", axis=1)

    # Reformats datetime columns
    for datetime_col in [
        "timestamp_created",
        "timestamp_updated",
        "author_last_played",
    ]:
        df_reviews[datetime_col] = pd.to_datetime(df_reviews[datetime_col], unit="s")


    # Coerces columns that should be ints to ints
    for int_col in [
        "recommendationid",
        "author_steamid",
        "steam_appid",
    ]:
        df_reviews[int_col] = df_reviews[int_col].astype(int)

    # Coerces column that should be float to float
    df_reviews["weighted_vote_score"] = df_reviews["weighted_vote_score"].astype(float)

    # Sets index to recommendation id column
    df_reviews = df_reviews.set_index("recommendationid")

    return df_reviews


In [6]:
def ingest_reviews(paths, meta, i):

    # Reads review jsons into a dask dataframe
    ddf_review_jsons = dd.read_json(paths, include_path_column=True)

    # Concatenates values of path column to just name of containing folder.
    # (Review download function uses steam appids as folder names)
    ddf_review_jsons["path"] = (
        ddf_review_jsons["path"].astype(str).str[:-14].replace(".+/", "", regex=True)
    )

    # Extracts reviews into dask DataFrame of pandas DataFrames (using template)
    ddf_reviews_dfs = ddf_review_jsons.apply(extract_reviews, axis=1, meta=meta)

    # Concatenates dask DataFrame of review DataFrames into a dask DataFrame of reviews
    ddf_reviews = dd.concat(ddf_reviews_dfs.compute().tolist())
    clear_output()
    print(f"Batch {i+1} reviews loaded!")

    ddf_reviews = ddf_reviews.repartition(npartitions=1)

    # Writes reviews to parquet file
    try:
        ddf_reviews.to_parquet("data\\reviews.parquet", append=(i > 0))
        clear_output()
        print(f"Succesffully imported batch {i+1}")
        return True, meta
    except:
        clear_output()
        print("Error! Bad meta.\nUpdating...")
        meta = dd.utils.make_meta(ddf_reviews)
        meta.to_parquet("dask_templates\\reviews_meta.parquet")
        return False, dd.utils.make_meta(ddf_reviews)

In [ ]:
# Imports template dataframe so dask knows to to interpret things
meta = dd.utils.make_meta(pd.read_parquet("dask_templates\\reviews_meta.parquet"))

In [ ]:
ingest_finished = False
is_meta_valid = True

while ingest_finished == False:
    for i, paths_subset in enumerate(np.array_split(paths, len(paths)/2**13)):
        if is_meta_valid:
            new_meta = ingest_reviews(paths_subset.tolist(), meta, i)
        else:
            break
            
    if is_meta_valid:
        ingest_finished = True
        client.shutdown()
        clear_output()
        print('Ingest finished!')
    else:
        meta = new_meta

Ingest finished!
